In [ ]:
# import libraries
import os
from os import listdir
import pandas as pd
import numpy as np
from numpy import asarray
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
import cv2
import secrets

# FIRST TASK: CLASSIFY FACES FROM NONE FACES

In [ ]:
# image dataset path
dataset = 'faceornot/'

In [ ]:
# check image names
for picture in listdir(dataset) :
     if picture != 'Thumbs.db':
            print(picture)#print(picture[0])
            

In [ ]:
# function that creates features and targets
def load_images(directory):
    train_pictures = list()
    targets = list()
    gray_scaled = list()
    images = pd.DataFrame()
    print('Converting to numpy array ...')
    for picture in listdir(directory) :
         if picture != 'Thumbs.db':
                 # load the picture from directory
                photo = load_img(directory + picture, target_size = (224,224))
                
                #convert image to numpy array
                photo = img_to_array(photo, dtype='uint8')  
                
                #photo = photo.reshape(1, 224, 224, 3)
                
                # append to list
                #train_pictures.append(photo)
                
                # convert image to grayscale
                gray_scale = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)
                gray_scaled.append(gray_scale)
                
                # label targets
                if picture[0] == 'f':
                    targets.append(1)
                else:
                    targets.append(0)
    print(' ')
    #print('Converting to Pandas dataframe ...') 

    #images['image_arrays'] = gray_scaled
    #images['targets'] = targets
    
    X_TRAIN = asarray(gray_scaled, dtype = 'uint8')
    Y_TRAIN = asarray(targets, dtype = 'uint8')
    print(' ')
    print('Done!')
    return X_TRAIN, Y_TRAIN #images

In [ ]:
# calling the function
X,Y = load_images(dataset) #images = load_images(dataset)

In [ ]:
plt.imshow(X[1])

In [ ]:
plt.imshow(X[47])

#### Reshape the data

In [ ]:
X.shape

In [ ]:
X = np.array(X).reshape((80, 224*224))

In [ ]:
X.shape

#### Split the data

In [ ]:
# split the dataset
from sklearn.model_selection import train_test_split
x_train, x_test, y_train,y_test = train_test_split(X,Y, test_size = 0.2, random_state = 3)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
#from sklearn.preprocessing import StandardScaler
#sc_x = StandardScaler()
#x_train = sc_x.fit_transform(x_train)
#x_test = sc_x.transform(x_test)

### Reducing the dimension of the Images

In [ ]:
# Application of PCA
from sklearn.decomposition import PCA
pca = PCA()  #kernel ='rbf' # this takes the indpendent variables that interpret the dataset the best.

In [ ]:
x_train_check = pca.fit(x_train)  #x_train_check = pca.fit_transform(x_train) 

In [ ]:
explained_variance = pca.explained_variance_ratio_
explained_variance # this was used to know how many components the PCA would explain "the most the variance".


In [ ]:
cumsum = np.cumsum(explained_variance) # find the cummulative summation of that array
cumsum

In [ ]:
n_comp = np.argmax(cumsum >= 0.96) + 1 # return the indices in the cumsum with a value greater than 0.96 ... this value is 40( this is 6
# because of pythons indexing, then add plus 1: 41. 

In [ ]:
n_comp

In [ ]:
pca = PCA(n_components = n_comp)
x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)

In [ ]:
# Basically the eigen faces are the vectors that best represent the large image matrices

In [ ]:
eigenfaces = pca.components_.reshape((n_comp, 224, 224))

In [ ]:
plt.imshow(eigenfaces[0])

In [ ]:
plt.imshow(eigenfaces[1])

In [ ]:
plt.imshow(eigenfaces[2])

In [ ]:
# let us convert back to the original images from the compressed format.
X_recover = pca.inverse_transform(x_train)
X_recover= X_recover.reshape((64,224,224))

In [ ]:
plt.imshow(X_recover[0])

In [ ]:
plt.imshow(X_recover[1])

In [ ]:
plt.imshow(X_recover[2])

#### Classifying using Support Vector Machines

In [ ]:
from sklearn.svm import SVC
Fclassifier = SVC()
#Fclassifier.fit(x_train, y_train)

In [ ]:
# Using grid search to obtain optimal hyperparameters 
from sklearn.model_selection import GridSearchCV
parameters = [{'C' : [1,100,1000], 'kernel':['linear']},
              {'C' : [1,100,1000], 'kernel':['rbf'], 'gamma':[0.5,0.1, 0.01,0.001,0.0001]}]

grid_search = GridSearchCV(estimator = Fclassifier, 
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs =-1)
grid_search = grid_search.fit(x_train, y_train)

In [ ]:
best_accuracy = grid_search.best_score_
best_accuracy * 100 # Multiplied by 100 to get in percentage 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

parameters = [{'C' : [1,100,1000], 'kernel':['linear']},
              {'C' : [1,100,1000], 'kernel':['rbf'], 'gamma':[0.5,0.1, 0.01,0.001,0.0001]}]


random_search =  RandomizedSearchCV(Fclassifier,parameters,n_iter =100, random_state = 0, verbose = 1)

random_search = random_search.fit(x_train, y_train)

In [ ]:
best_accuracy = random_search.best_score_
best_accuracy * 100

In [ ]:
# best parameters for randomsearch
best_parametersR = random_search.best_params_
best_parametersR

In [ ]:
# best parameters for grid search
best_parameters = grid_search.best_params_
best_parameters

In [ ]:
Fclassifier = grid_search.best_estimator_
Fclassifier.fit(x_train, y_train)#.fit(x_train, y_train)

In [ ]:
y_pred = Fclassifier.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cn = confusion_matrix(y_test, y_pred)

In [ ]:
print(cn)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
print(accuracy_score(y_pred, y_test) * 100)

In [ ]:
# analysis

In [ ]:
# What does the x_test look like?
x_test

In [ ]:
x_test.shape

In [ ]:
test_recover = pca.inverse_transform(x_test)

In [ ]:
test_recover = test_recover.reshape((16,224,224))

In [ ]:
plt.imshow(test_recover[0])

In [ ]:
plt.imshow(test_recover[1])

In [ ]:
plt.imshow(test_recover[2])

In [ ]:
plt.imshow(test_recover[3])

In [ ]:
plt.imshow(test_recover[4])

In [ ]:
plt.imshow(test_recover[5])

In [ ]:
print(y_pred)

In [ ]:
print(y_test)

# SECOND TASK: IDENTIFY WHO IS IN THE IMAGE

In [ ]:
# image dataset path
dataset = 'whoisthis/'

In [ ]:
name_file = pd.read_csv('name_tags.csv')

In [ ]:
name_file.head()

In [ ]:
name_file.shape

In [ ]:
def names(name_file):
    name_tag= dict()
    for i in range(len(name_file)):
        name, tags = name_file["image_name"][i], name_file['tags'][i]
        name_tag[name] = tags
    return(name_tag)


In [ ]:
see = names(name_file)

In [ ]:
see

In [ ]:
def name_passcode(list_):
    
    maps = dict() 
    for i  in list_:
        maps[i] = secrets.token_hex(24)
    return maps

In [ ]:
def load_images_targets(directory, name_file):
    #train_pictures = list()
    targets = list()
    gray_scaled = list()
    name_targ = names(name_file)
    images = pd.DataFrame()
    passcode_targets = list()
    print('Converting to numpy array ...')
    for picture in listdir(directory) :
         if picture != 'Thumbs.db':
                 # load the picture from directory
                photo = load_img(directory + picture, target_size = (224,224))
                
                #convert image to numpy array
                photo = img_to_array(photo, dtype='uint8')  
                
                #photo = photo.reshape(1, 224, 224, 3)
                
                # append to list
                #train_pictures.append(photo)
                
                # convert image to grayscale
                gray_scale = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)
                gray_scaled.append(gray_scale)
                
                # label targets
                targets.append(name_targ[picture.split('.')[0]])

    print(' ')
    maps = name_passcode(targets)
    
    for i in targets:
        passcode_targets.append(maps[i])
        
    
        
    
    X_TRAIN = asarray(gray_scaled, dtype = 'uint8')
    Y_TRAIN = asarray(passcode_targets)#, dtype = 'uint8')
    print(' ')
    print('Done!')
    return X_TRAIN, Y_TRAIN, maps #images

In [ ]:
X,Y, mapper = load_images_targets(dataset, name_file)

#### RESHAPE THE DATA

In [ ]:
mapper

In [ ]:
for keys in mapper:
    print(keys, '==>', mapper[keys])

In [ ]:
# creating an inverse mapper
inv_mapper = dict()
for keys in mapper:
    inv_mapper[mapper[keys]] = keys
    
   

In [ ]:
inv_mapper

In [ ]:
Y[:5]

In [ ]:
X.shape

In [ ]:
X_r = np.array(X).reshape((150, 224*224))

In [ ]:
X_r.shape

#### SPLIT THE DATA

In [ ]:
# split the dataset
from sklearn.model_selection import train_test_split
x_train, x_test, y_train,y_test = train_test_split(X_r,Y, test_size = 0.2, random_state = 3)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

#### REDUCING THE DIMENSION OF THE IMAGES

In [ ]:
# Application of PCA
from sklearn.decomposition import PCA
pca = PCA()  #kernel ='rbf' # this takes the indpendent variables that interpret the dataset the best.

In [ ]:
x_train_check = pca.fit(x_train)

In [ ]:
explained_variance = pca.explained_variance_ratio_
explained_variance # this was used to know how many components the PCA would explain "the most the variance".


In [ ]:
cumsum1 = np.cumsum(explained_variance) # find the cummulative summation of that array
cumsum1

In [ ]:
n_comp = np.argmax(cumsum1 >= 0.99) + 1 

In [ ]:
n_comp

In [ ]:
pca1= PCA(n_components = n_comp)
x_train = pca1.fit_transform(x_train)
x_test = pca1.transform(x_test)

In [ ]:
eigenfaces = pca1.components_.reshape((n_comp, 224, 224))

In [ ]:
plt.imshow(eigenfaces[0])

In [ ]:
plt.imshow(eigenfaces[1])

In [ ]:
plt.imshow(eigenfaces[2])

In [ ]:
# let us convert back to the original images from the compressed format.
X_recover = pca1.inverse_transform(x_train)
X_recover= X_recover.reshape((120,224,224))

In [ ]:
plt.imshow(X_recover[0])

In [ ]:
plt.imshow(X_recover[1])

In [ ]:
plt.imshow(X_recover[2])

#### CLASSIFYING USING SUPPORT VECTOR MACHINES

In [ ]:
from sklearn.svm import SVC
Wclassifier = SVC(class_weight='balanced')

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters =   [{'C' : [1,100,1000], 'kernel':['linear']},
              {'C' : [1,100,1000], 'kernel':['rbf'], 'gamma':[0.5,0.1, 0.01,0.001,0.0001]}]

             

grid_search = GridSearchCV(estimator = Wclassifier, 
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 6,
                           n_jobs =-1)
grid_search = grid_search.fit(x_train, y_train)

In [ ]:
best_accuracy = grid_search.best_score_
best_accuracy * 100 # Multiplied by 100 to get in percentage 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

parameters = [{'C' : [1,100,1000], 'kernel':['linear']},
              {'C' : [1,100,1000], 'kernel':['rbf'], 'gamma':[0.5,0.1, 0.01,0.001,0.0001]}]


random_search =  RandomizedSearchCV(Wclassifier,parameters,n_iter =100, random_state = 0, verbose = 1)

random_search = random_search.fit(x_train, y_train)

In [ ]:
best_accuracy = random_search.best_score_
best_accuracy * 100

In [ ]:
# best parameters for randomsearch
best_parametersR = random_search.best_params_
best_parametersR

In [ ]:
# best parameters for gridsearch
best_parametersG = grid_search.best_params_
best_parametersG

In [ ]:
Wclassifier = grid_search.best_estimator_
Wclassifier.fit(x_train, y_train)

In [ ]:
y_pred = Wclassifier.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cn = confusion_matrix(y_test, y_pred)

In [ ]:
print(cn)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
print(accuracy_score(y_pred, y_test) * 100)

## Using SGDClassifier as a linear Support Vector Machine classifier.

In [ ]:
# A stochastic gradient descent with hyper-parameters such as loss and penality set to 'hinge' and 'l2' i.e ridge regularization
# respectively is equivalent to a linear support vector machines classifier.

# This is important because of the online learning capabilty (i.e .partial_fit() ) the SGDClassifier provides

from sklearn.linear_model import SGDClassifier
Wclassifier2 = SGDClassifier(loss="hinge", penalty="l2") 

In [ ]:
Wclassifier2.partial_fit(x_train, y_train, classes=np.unique(y_train))

# This link explains the use of classes above: https://stackoverflow.com/questions/42147302/sklearn-sgd-partial-fit

In [ ]:
y_pred2 = Wclassifier2.predict(x_test)

In [ ]:
cn2 = confusion_matrix(y_test, y_pred2)

In [ ]:
print(cn2)

In [ ]:
print(classification_report(y_test, y_pred2))

In [ ]:
print(accuracy_score(y_pred2, y_test) * 100)

In [ ]:
# When using .fit() both models perform exactly the same, but when using .partial_fit() it performs lower than the normal svm. 
# This is a problem more data can solve.

### Visualising the predictions for the normal SVM Classifier.

In [ ]:
# lets convert back to the names
y_test =[inv_mapper[i] for i in y_test]
y_pred =[inv_mapper[i] for i in y_pred]


In [ ]:
zipped = zip(y_pred, y_test)
print('Predicted | Actual')
print('-------------------')
for l1,l2 in zipped:
    print(f'- {l1} | {l2} ')

### Visualising the predictions for the SGD Classifier(linear SVM Classifier).

In [ ]:
y_pred2 =[inv_mapper[i] for i in y_pred2]


In [ ]:
zipped = zip(y_pred2, y_test)
print('Predicted | Actual')
print('-------------------')
for l1,l2 in zipped:
    print(f'- {l1} | {l2} ')

# USING LBPHFACERECOGINIZER

#### LBPH is used to see if the model will perform a lot better than SVC on a the given dataset . (maybe like 30+ increase in accuracy)

In [ ]:
def load_images_targets(directory, name_file):
    #train_pictures = list()
    targets = list()
    gray_scaled = list()
    name_targ = names(name_file)
    print('Converting to numpy array ...')
    for picture in listdir(directory) :
         if picture != 'Thumbs.db':
                 # load the picture from directory
                photo = load_img(directory + picture, target_size = (224,224))
                
                #convert image to numpy array
                photo = img_to_array(photo, dtype='uint8')  
                
                #photo = photo.reshape(1, 224, 224, 3)
                
                # append to list
                #train_pictures.append(photo)
                
                # convert image to grayscale
                gray_scale = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)
                gray_scaled.append(gray_scale)
                
                # label targets
                targets.append(name_targ[picture.split('.')[0]])


            
    X_TRAIN = asarray(gray_scaled, dtype = 'uint8')
    Y_TRAIN = asarray(targets)#, dtype = 'uint8')
    print(' ')
    print('Done!')
    return X_TRAIN, Y_TRAIN #images

In [ ]:
X,Y = load_images_targets(dataset, name_file)

In [ ]:
# The  LBPHFACERECOGINIZER does not take categorical targets so I converted to integers.
hold = Y.tolist() # convert array to list
uniq = set(hold) # remove duplicates
names = list(uniq) # convert to list

In [ ]:
mapped = {names[i]:i for i in range(len(names))} # creat dictionary mapping names to numbers
inv_mapped = {i:names[i] for i in range(len(names))}

In [ ]:
mapped

In [ ]:
# create a new Y
Y_i = [mapped[i] for i in hold]    

In [ ]:
# To make the targets and the feature are still mapped correctly Lets check the first, last and any random image

# The first
plt.imshow(X[0])

In [ ]:
Y_i[0]

In [ ]:
# The Second
plt.imshow(X[43])

In [ ]:
Y_i[43]

In [ ]:
# The Last
plt.imshow(X[119])

In [ ]:
Y_i[119]

In [ ]:
# Split the data
x_train, x_test, y_train,y_test = train_test_split(X,Y_i, test_size = 0.2, random_state = 3)

In [ ]:
x_train.shape

In [ ]:
Y_i[:5]

In [ ]:
x_train.shape

In [ ]:
#x_train = x_train.reshape((120, 224, 224))

In [ ]:
x_train.shape

In [ ]:
clf = cv2.face.LBPHFaceRecognizer_create()
clf.train(x_train, np.array(y_train))

In [ ]:
x_test.shape

In [ ]:
y_pred = list()
for i in x_test:
    y_p,_ = clf.predict(i)
    y_pred.append(y_p)

In [ ]:
cn = confusion_matrix(y_test, y_pred)

In [ ]:
cn

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
print(accuracy_score(y_pred, y_test) * 100)

In [ ]:
y_pred = [inv_mapped[i] for i in y_pred]
y_test = [inv_mapped[i] for i in y_test]

In [ ]:
zipped = zip(y_pred, y_test)
print('Predicted | Actual')
print('-------------------')
for l1,l2 in zipped:
    print(f'- {l1} | {l2} ')
   

### It just increased by plus 7, which is not impressive. Best solution is get more data. Possibly +150 images per face

###### REFERENCES

In [ ]:
#  https://www.kaggle.com/hamishdickson/preprocessing-images-with-dimensionality-reduction

#  https://scikit-learn.org/stable/auto_examples/applications/plot_face_recognition.html

#  https://scikit-learn.org/stable/modules/sgd.html#sgd

# https://shankarmsy.github.io/posts/pca-sklearn.html